In [39]:
import requests
import bs4 as bs

mozilla_agent = "Mozilla/5.0 (Windows NT 6.1; WOW64; rv:47.0) Gecko/20100101 Firefox/47.0"
headers = {'User-Agent': mozilla_agent}

toc_url = 'https://practicalguidetoevil.wordpress.com/table-of-contents/'
toc_response = requests.get(toc_url, headers=headers)
toc_html = bs.BeautifulSoup(toc_response.content, 'html.parser')
toc_links = toc_html.findAll('li')

whitelist = ['chapter', 'epilogue', 'prologue', 'interlude']
scrape_urls = []

for link in toc_links:
    try:
        url = link.a.get('href')
    except:
        pass

    matches = [white_word in url for white_word in whitelist]
    if any(matches):
        scrape_urls += [url]

chapters = []
for url in scrape_urls:
    response = requests.get(url, headers=headers)
    bs_html = bs.BeautifulSoup(response.content, 'html.parser')
    article = bs_html.findAll('article')[0]   
    chapter_title = article.h1.text

    # Get all article text
    raw = article.text
    # cut first and last lines (author, follow me)
    lines = raw.split('\n')[8:-7]
    chapter_html = ''
    for line in lines:
        chapter_html += f'<p>{line}</p>\n'
    chapters += [(chapter_title, chapter_html)]
    print(chapter_title)

Prologue
Chapter 1: Knife
Chapter 2: Invitation
Chapter 3: Party
Chapter 4: Name
Chapter 5: Role
Chapter 6: Aspect
Chapter 7: Sword
Chapter 8: Introduction
Chapter 9: Claimant
Chapter 10: Menace
Chapter 11: Sucker Punch
Chapter 12: Squire
Chapter 13: Order
Chapter 14: Villain
Chapter 15: Company
Chapter 16: Game
Chapter 17: Set
Chapter 18: Match
Chapter 19: Pivot
Chapter 20: Rise
Chapter 21: Fall
Chapter 22: All According To
Chapter 23: Morok’s Plan
Chapter 24: Aisha’s Plan
Chapter 25: Snatcher’s Plan
Chapter 26: Juniper’s Plan
Chapter 27: Callow’s Plan
Chapter 28: Win Condition
Epilogue
Prologue
Prologue
Chapter 1: Supply
Chapter 2: Demand
Chapter 3: Cost
Heroic Interlude: Balestra
Chapter 4: Return
Chapter 5: Recognition
Chapter 6: Rapport
Chapter 7: Reception
Chapter 8: Reversal
Chapter 9: Rematch
Chapter 10: Release
Chapter 11: Report
Chapter 12: Reproval
Heroic Interlude: Riposte
Villainous Interlude: Coulisse
Chapter 13: Fireside
Chapter 14: Situation
Chapter 15: Council
Chapter 

Chapter 58: Prolong
Chapter 59: Review
Chapter 60: Melancholy
Chapter 61: Reformation
Chapter 62: Pledged
Chapter 63: Draft
Chapter 64: Breathe
Chapter 65: Convivial
Chapter 66: Silvered
Chapter 67: Starlight
Chapter 68: Apropos
Chapter 69: Repute
Chapter 70: Dawning
Chapter 71: Verge
Interlude: Iron
Interlude: Rope
Interlude: Candle
Interlude: Harp
Interlude: Bone
Interlude: Mirror
Interlude: And Yet We Stand
Chapter 72: Rumours
Chapter 73: Discerning
Chapter 74: Partial
Chapter 75: Analog
Chapter 76: Procession
Chapter 77: Artless
Chapter 78: Trenchant
Chapter 79: Hitch
Chapter 80: Descant
Chapter 81: Devotional
Chapter 82: Delegations
Chapter 83: A Mould Unbroken
Interlude: Rise, Rise
Interlude: All Ye Villains
Interlude: So Smile, Tyrants
Interlude: And So Let Us Be
Interlude: Wicked
Interlude: Suffer No Compromise In This
Interlude: A Hundred Battles
Chapter 84: Declaration
Chapter 85: When It Rains
Chapter 86: It Pours
Chapter 87: Connive
Chapter 88: Testament
Chapter 89: Sing We

In [41]:
from ebooklib import epub
book = epub.EpubBook()

# Metadata
book.set_identifier('id00000')
book.set_title('A Practical Guide to Evil')
book.set_language('en')
book.add_author('Erraticerrata')

spine = ['nav']
chapter_counter = 0
chapter_objects = []
# Chapters
for chapter_title, chapter_html in chapters:
    file_name = f'chap_{chapter_counter}.xhtml'
    chapter = epub.EpubHtml(title=chapter_title, file_name=file_name, lang='en')
    chapter.set_content(f'<html><body><h1>{chapter_title}</h1>{chapter_html}</body></html>')
    book.add_item(chapter)
    spine += [chapter]
    
    chapter_counter += 1
    chapter_objects += [chapter]
    
book.toc = ((epub.Section('PGTE'), chapter_objects), )

# basic spine
book.spine = spine

style = 'body { font-family: Times, Times New Roman, serif; }'

nav_css = epub.EpubItem(uid="style_nav",
                        file_name="style/nav.css",
                        media_type="text/css",
                        content=style)
book.add_item(nav_css)

book.add_item(epub.EpubNcx())
book.add_item(epub.EpubNav())

epub.write_epub('pgte.epub', book, {})